# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
!pip -q install ./python

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="./Navigation_Environment/Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [2]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

NameError: name 'env' is not defined

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [1]:
from unityagents import UnityEnvironment
import numpy as np

env = UnityEnvironment(file_name="./Navigation_Environment/Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [2]:
# Load unity Environment
import time
from collections import namedtuple

import numpy as np
import torch
from DeepQAgent.Agent import ExpBuffer, Agent
from DeepQAgent.QNetwork import QNetwork

# experience = namedtuple('experience', ['state', 'action', 'reward', 'next_state', 'done'])
# experiences = []

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

env_info = env.reset(train_mode=False)[brain_name] # reset the environment

action_size = brain.vector_action_space_size       # get the size of the action space
state = env_info.vector_observations[0]            # get the current state
state_size = len(state)
score = 0                                          # initialize the score
seed = 0

qnet = QNetwork(len(state), action_size, 0)
buff = ExpBuffer(200, 0, torch.device('cuda:0' if torch.cuda.is_available() else 'cpu:0'))

agent = Agent( state_size, action_size, seed,
               torch.device('cuda:0' if torch.cuda.is_available() else 'cpu:0')
             )

start_time = time.time()
t = 0

while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score

    est = qnet(torch.from_numpy(state).float() ).detach().cpu().max().item()
    tar = qnet(torch.from_numpy(next_state).float() ).detach().cpu().max().item()
    
    buff.add_exp( target = tar,
                  estimate = est,
                  state = state, action = action, reward = reward,
                  next_state = next_state, done = done
                )
    
    
    state = next_state                             # roll over the state to next time step
    
    # inspect the Experience tuples:
    # print("\rstate: ",state," | action: ",action," | reward: ", reward," | next_state: ", next_state, " | done: ", done, end = "")    
    # experiences.append( experience(state, action, reward, next_state, done) ) # store the experience tuples
    
    """
    log = "\rTimesteps: {}, Reward: {}, Done: {}, Est: {}, Tar: {} +> {}".format(t, reward, done, est.data.numpy(),
                                                                                 tar.data.numpy(),
                                                                                 (abs(est - tar) + 0.0005)**0.95
                                                                                )
    """

    log = "\rTimesteps: {}, Reward: {}, Done: {}".format( t, reward, done )
    
    log = log + " "*(100 - len(log))
    print(log, end = "\r")
          
    
    if done:                                       # exit loop if episode finished
        break
    
    t += 1                                         # update timestep counter
    
    
end_time = time.time()

print("\nScore: {}".format(score))

print("Total Time taken: {} seconds".format(end_time - start_time))

# Close the Environment post completion
env.close()

Timesteps: 299, Reward: 0.0, Done: True                                                            
Score: 1.0
Total Time taken: 30.120036840438843 seconds


In [3]:
# env.close()

In [4]:
len(buff) # experiences[0]

300

In [5]:
s, a, r, n_s, d, p = buff.sample()

In [6]:
p[:10]# p

tensor([[0.0028],
        [0.0042],
        [0.0025],
        [0.0016],
        [0.0018],
        [0.0072],
        [0.0038],
        [0.0028],
        [0.0039],
        [0.0078]], device='cuda:0')

In [7]:
from DeepQAgent.Agent import ExpBuffer, Agent
from DeepQAgent.QNetwork import QNetwork

epsilon = 1.0
for idx, sample_state in enumerate(s[:200]):
    
    sample_state = sample_state.detach().cpu().data.numpy()
    t_action, t_estimate = agent.act(sample_state, epsilon);
    # print(t_action, t_estimate, t_estimate.cpu().data.numpy()[0][t_action])

    # get the reward, action, dones -> take the next state
    t_target = r[idx]+agent.qnet_target( n_s[idx] ).detach().cpu().max()
    t_estimate = t_estimate.cpu().data.numpy()[0][t_action]
    # print(t_target.cpu().data.numpy()[0], t_estimate)
    
    """
    s[idx] = s[idx].detach().cpu().data.numpy()
    r[idx] = r[idx].detach().cpu().data.numpy()
    n_s[idx] = n_s[idx].detach().cpu().data.numpy()
    d[idx] = d[idx].detach().cpu().data.numpy()
    """
    # print(n_s[idx])
    
    agent.step( t_target.cpu().data.numpy()[0], t_estimate,
                s[idx].cpu().data.numpy(), t_action, r[idx].cpu().data.numpy(),
                n_s[idx].cpu().data.numpy(), d[idx].cpu().data.numpy()
              )
    
    #agent.step( sample_state )
    epsilon *= 0.99995

Triggering Learn | Buffer Size: 128 | t: 0
tensor(0.0170, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 136 | t: 0
tensor(0.0132, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 144 | t: 0
tensor(0.0017, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 152 | t: 0
tensor(0.0023, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 160 | t: 0
tensor(0.0013, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 168 | t: 0
tensor(0.0011, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 176 | t: 0
tensor(0.0008, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 184 | t: 0
tensor(0.0008, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 192 | t: 0
tensor(0.0009, device='cuda:0', grad_fn=<MeanBackward0>)
Triggering Learn | Buffer Size: 200 | t: 0
tensor(0.0010, device='cuda:0', grad_fn=<MeanBackward0>)


In [8]:
a = torch.from_numpy( np.array([i for i in range(12, 18)]) )

In [10]:
a

tensor([12, 13, 14, 15, 16, 17], dtype=torch.int32)

In [15]:
a = a**2
a

tensor([144, 169, 196, 225, 256, 289], dtype=torch.int32)

In [16]:
b = torch.from_numpy( np.array([i for i in range(2, 8)]) )
b

tensor([2, 3, 4, 5, 6, 7], dtype=torch.int32)

In [17]:
a*b

tensor([ 288,  507,  784, 1125, 1536, 2023], dtype=torch.int32)

In [22]:
a.resize_(1, a.shape[0])
a

tensor([[144, 169, 196, 225, 256, 289]], dtype=torch.int32)

In [26]:
torch.mean(a.float())

tensor(213.1667)

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [16]:
import torch
import numpy as np

a = torch.from_numpy( np.array([1.0, -1.0, 0.657, 0.7, 0.8, 2.0, -10.0]) )
a

tensor([  1.0000,  -1.0000,   0.6570,   0.7000,   0.8000,   2.0000, -10.0000],
       dtype=torch.float64)

In [22]:
torch.nn.functional.leaky_relu(a, negative_slope = 0.02)

tensor([ 1.0000, -0.0200,  0.6570,  0.7000,  0.8000,  2.0000, -0.2000],
       dtype=torch.float64)

In [4]:
import torch
import torch.optim as optim
import numpy as np
import os


from DeepQAgent.QNetwork import QNetwork

In [5]:
trailNet = QNetwork(37, 4, 0)

In [23]:
tmp = [1.        , 0.        , 0.        , 0.        , 0.84173167, 0.        , 0.        , 1.        , 0.        , 0.06433759,
       0.        , 1.        , 0.        , 0.        , 0.2517986 , 1.        , 0.        , 0.        , 0.        , 0.72478515,
       0.        , 1.        , 0.        , 0.        , 0.2527748 , 0.        , 0.        , 1.        , 0.        , 0.08042004,
       0.        , 1.        , 0.        , 0.        , 0.31255436, 0.        , 7.81049442
      ]

next_tmp = torch.from_numpy(np.array(tmp)).float()
next_tmp

tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.8417, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0643, 0.0000, 1.0000, 0.0000, 0.0000, 0.2518, 1.0000, 0.0000, 0.0000,
        0.0000, 0.7248, 0.0000, 1.0000, 0.0000, 0.0000, 0.2528, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0804, 0.0000, 1.0000, 0.0000, 0.0000, 0.3126, 0.0000,
        7.8105])

In [9]:
next_tmp.resize_(1, next_tmp.shape[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.8417, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0643, 0.0000, 1.0000, 0.0000, 0.0000, 0.2518, 1.0000, 0.0000, 0.0000,
         0.0000, 0.7248, 0.0000, 1.0000, 0.0000, 0.0000, 0.2528, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0804, 0.0000, 1.0000, 0.0000, 0.0000, 0.3126, 0.0000,
         7.8105]])

In [1]:
import torch
from DeepQAgent.Agent import ExpBuffer
from DeepQAgent.QNetwork import QNetwork
import numpy as np

trial_agent = ExpBuffer( 64, 0, torch.device('cuda:0' if torch.cuda.is_available() else 'cpu:0') )

In [2]:
tmp = [1., 0., 0., 0., 0.84173167, 0., 0., 1., 0., 0.06433759,
       0., 1., 0., 0., 0.2517986 , 1., 0., 0., 0., 0.72478515,
       0., 1., 0., 0., 0.2527748 , 0., 0., 1., 0., 0.08042004,
       0., 1., 0., 0., 0.31255436, 0., 7.81049442
      ]

tmp = torch.from_numpy(np.array(tmp)).float()
next_tmp = torch.from_numpy(np.array(tmp)).float()
next_tmp

tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.8417, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0643, 0.0000, 1.0000, 0.0000, 0.0000, 0.2518, 1.0000, 0.0000, 0.0000,
        0.0000, 0.7248, 0.0000, 1.0000, 0.0000, 0.0000, 0.2528, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0804, 0.0000, 1.0000, 0.0000, 0.0000, 0.3126, 0.0000,
        7.8105])

In [3]:
tmp = torch.from_numpy( np.array(tmp) ).resize_(1, len(tmp)).float()
tmp

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.8417, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0643, 0.0000, 1.0000, 0.0000, 0.0000, 0.2518, 1.0000, 0.0000, 0.0000,
         0.0000, 0.7248, 0.0000, 1.0000, 0.0000, 0.0000, 0.2528, 0.0000, 0.0000,
         1.0000, 0.0000, 0.0804, 0.0000, 1.0000, 0.0000, 0.0000, 0.3126, 0.0000,
         7.8105]])

In [4]:
next_tmp = next_tmp.float()
next_tmp

tensor([1.0000, 0.0000, 0.0000, 0.0000, 0.8417, 0.0000, 0.0000, 1.0000, 0.0000,
        0.0643, 0.0000, 1.0000, 0.0000, 0.0000, 0.2518, 1.0000, 0.0000, 0.0000,
        0.0000, 0.7248, 0.0000, 1.0000, 0.0000, 0.0000, 0.2528, 0.0000, 0.0000,
        1.0000, 0.0000, 0.0804, 0.0000, 1.0000, 0.0000, 0.0000, 0.3126, 0.0000,
        7.8105])

In [5]:
tmp.shape, next_tmp.shape

(torch.Size([1, 37]), torch.Size([37]))

In [6]:
qvalues = QNetwork(37,4,0)(tmp)
qvalues

tensor([[ 0.0953, -0.1096,  0.0913, -0.0791]], grad_fn=<AddmmBackward>)

In [7]:
trial_agent.add_sample(target = 1.0, estimate = qvalues.max(1)[0], state = tmp, action = 1, reward = 100, next_state = next_tmp, done = False)
trial_agent.add_sample(target = 1.0, estimate = qvalues.min(1)[0], state = tmp, action = 2, reward = -100, next_state = next_tmp, done = False)

In [8]:
qvalues.min(1)[0]

tensor([-0.1096], grad_fn=<MinBackward0>)

In [9]:
trial_agent.get_priorities()

[tensor([0.4612]), tensor([0.5388])]

In [10]:
s, a, r, n_s, d = trial_agent.sample()

tensor([[1.0000, 0.0000, 0.0000,  ..., 0.3126, 0.0000, 7.8105],
        [1.0000, 0.0000, 0.0000,  ..., 0.3126, 0.0000, 7.8105],
        [1.0000, 0.0000, 0.0000,  ..., 0.3126, 0.0000, 7.8105],
        ...,
        [1.0000, 0.0000, 0.0000,  ..., 0.3126, 0.0000, 7.8105],
        [1.0000, 0.0000, 0.0000,  ..., 0.3126, 0.0000, 7.8105],
        [1.0000, 0.0000, 0.0000,  ..., 0.3126, 0.0000, 7.8105]],
       device='cuda:0')

In [23]:
z = torch.from_numpy(np.array([i + 0.5 for i in range(5)]))

In [24]:
z.requires_grad = False

In [4]:
a = np.array([i for i in range(10)])
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [11]:
a = [i for i in range(11, 20)]

In [13]:
np.random.choice(a)

17

In [15]:
import random
random.random()

0.24668202443497522

In [22]:
random.random()

0.3114221913826162

In [29]:
np.argmax(z.data.numpy())

4

In [31]:
a = [1, 2, 3, 4, 5, 6]
a

[1, 2, 3, 4, 5, 6]

In [32]:
del a[3]

In [33]:
a

[1, 2, 3, 5, 6]

In [63]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu:0")
a = np.array([i for i in range(1, 38)])
a.resize(1, a.shape[0])
a = torch.from_numpy(a).to(device)
a

tensor([[ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
         37]], device='cuda:0', dtype=torch.int32)

In [64]:
a.cpu().data.numpy().max(), a.shape

(37, torch.Size([1, 37]))

In [65]:
a = a.cpu().float()
a

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14.,
         15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
         29., 30., 31., 32., 33., 34., 35., 36., 37.]])

In [66]:
qvalues = QNetwork(37,4,0)(a)
qvalues

tensor([[ 0.5342, -0.2318,  0.0284,  0.2815]], grad_fn=<AddmmBackward>)

In [67]:
qvalues.data.numpy().max()

0.53423786

In [70]:
qvalues

tensor([[ 0.5342, -0.2318,  0.0284,  0.2815]], grad_fn=<AddmmBackward>)

In [75]:
np.random.choice(np.arange(qvalues.shape[1])).dtype

dtype('int32')

In [79]:
k = np.argmax(qvalues.cpu().data.numpy()).astype('int32')
k.dtype

dtype('int32')

In [56]:
np.round(qvalues.data.numpy().max(), 3)

0.534

In [57]:
a = [i for i in range(1, 11)]
a

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [62]:
k = 5
a[-k:], np.mean(a[-k:])

([6, 7, 8, 9, 10], 8.0)

In [3]:
from collections import deque, namedtuple

d = deque(maxlen = 10)
for i in range(10):
    d.append(1+i)
    
d

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [7]:
deque(sorted(d)), deque(sorted(d), maxlen = 5)

(deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), deque([6, 7, 8, 9, 10]))

In [23]:
exp = namedtuple('exp', ['f', 's'])
p = deque([exp( 10-i, i+1 ) for i in range(10)])
p

deque([exp(f=10, s=1),
       exp(f=9, s=2),
       exp(f=8, s=3),
       exp(f=7, s=4),
       exp(f=6, s=5),
       exp(f=5, s=6),
       exp(f=4, s=7),
       exp(f=3, s=8),
       exp(f=2, s=9),
       exp(f=1, s=10)])

In [24]:
p = sorted( p, key = lambda x: x.f)[-5:]
p

[exp(f=6, s=5), exp(f=7, s=4), exp(f=8, s=3), exp(f=9, s=2), exp(f=10, s=1)]

In [26]:
# int(3*1e5) - 102400
p = deque(p, maxlen = 10)
p

deque([exp(f=6, s=5),
       exp(f=7, s=4),
       exp(f=8, s=3),
       exp(f=9, s=2),
       exp(f=10, s=1)])

In [27]:
p.append(exp( 20, 1 ))
p.append(exp( 19, 2 ))
p.append(exp( 18, 3 ))
p.append(exp( 17, 4 ))
p.append(exp( 16, 5 ))
len(p)

10

In [28]:
p

deque([exp(f=6, s=5),
       exp(f=7, s=4),
       exp(f=8, s=3),
       exp(f=9, s=2),
       exp(f=10, s=1),
       exp(f=20, s=1),
       exp(f=19, s=2),
       exp(f=18, s=3),
       exp(f=17, s=4),
       exp(f=16, s=5)])

In [29]:
p.append(exp( 15, 4 ))

In [30]:
p

deque([exp(f=7, s=4),
       exp(f=8, s=3),
       exp(f=9, s=2),
       exp(f=10, s=1),
       exp(f=20, s=1),
       exp(f=19, s=2),
       exp(f=18, s=3),
       exp(f=17, s=4),
       exp(f=16, s=5),
       exp(f=15, s=4)])

In [31]:
int(6*1e3)

6000

In [29]:
import numpy as np
eps = 1.0
eps_decay = 0.99 # 0.985
for i in range(301):
    eps = eps * eps_decay
    if eps < 0.15:
        print(i+1)
        break
    print(np.round(eps, 3))

0.99
0.98
0.97
0.961
0.951
0.941
0.932
0.923
0.914
0.904
0.895
0.886
0.878
0.869
0.86
0.851
0.843
0.835
0.826
0.818
0.81
0.802
0.794
0.786
0.778
0.77
0.762
0.755
0.747
0.74
0.732
0.725
0.718
0.711
0.703
0.696
0.689
0.683
0.676
0.669
0.662
0.656
0.649
0.643
0.636
0.63
0.624
0.617
0.611
0.605
0.599
0.593
0.587
0.581
0.575
0.57
0.564
0.558
0.553
0.547
0.542
0.536
0.531
0.526
0.52
0.515
0.51
0.505
0.5
0.495
0.49
0.485
0.48
0.475
0.471
0.466
0.461
0.457
0.452
0.448
0.443
0.439
0.434
0.43
0.426
0.421
0.417
0.413
0.409
0.405
0.401
0.397
0.393
0.389
0.385
0.381
0.377
0.373
0.37
0.366
0.362
0.359
0.355
0.352
0.348
0.345
0.341
0.338
0.334
0.331
0.328
0.324
0.321
0.318
0.315
0.312
0.309
0.305
0.302
0.299
0.296
0.293
0.29
0.288
0.285
0.282
0.279
0.276
0.273
0.271
0.268
0.265
0.263
0.26
0.257
0.255
0.252
0.25
0.247
0.245
0.242
0.24
0.238
0.235
0.233
0.231
0.228
0.226
0.224
0.221
0.219
0.217
0.215
0.213
0.211
0.208
0.206
0.204
0.202
0.2
0.198
0.196
0.194
0.192
0.19
0.189
0.187
0.185
0.183
0.181
0.17

In [12]:
try:
    for i in range(int(1e6)):
        print("\r ", i, end = " ")
except KeyboardInterrupt:
    print ('KeyboardInterrupt exception is caught')
else:
    print ('No exceptions are caught')

  16902KeyboardInterrupt exception is caught  11894  


In [1]:
0.9**0.9

0.9095325760829622

In [2]:
0.9**2

0.81

In [9]:
0.9**0.001

0.9998946450345664